In [29]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from utils import write_log, get_device, process_data
from train import fit, evaluate
import CNN, DNN, MLP
import medmnist
from medmnist import INFO, Evaluator

## We work on the 2D dataset with size 28x28

In [30]:
data_flag = 'bloodmnist'
download = True
BATCH_SIZE = 128

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

## First, we read the MedMNIST data, preprocess them and encapsulate them into dataloader form.

In [31]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)
validation_dataset = DataClass(split='val', transform=data_transform, download=download)

write_log('train_dataset_log.txt', str(train_dataset))
write_log('test_dataset_log.txt', str(test_dataset))
write_log('validation_dataset_log.txt', str(validation_dataset))

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
validation_loader = data.DataLoader(dataset=validation_dataset, batch_size=2*BATCH_SIZE, shuffle=False)


In [ ]:
# variables declaration
model_type = 'CNN'
n_epochs = 20
n_layers = 1
n_inputs = n_channels * 28 * 28
size_hidden_layer = (n_inputs + n_classes) // 2 # n_classes = n_outputs
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001


device = get_device()

assert device is not None

write_log('main_log.txt', f'Using device: {device}\n')

assert model_type is not None

if model_type == 'CNN':
    model = CNN.CNN(n_channels, n_classes)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    X, y = process_data(train_loader, flag=False)
    loss_values_training, trained_model = fit(device, X, y, model, loss_function, optimizer, n_epochs, batch_size=BATCH_SIZE)
    write_log('loss_values_training.txt', str(loss_values_training))

    # evaluate data
    X, y = process_data(validation_loader, flag=False)
    CM_val, f1_val = evaluate(device, X, y, trained_model, batch_size=BATCH_SIZE)
    write_log('validation_results.txt', f'Confusion Matrix:\n{CM_val}\nF1 Score: {f1_val}\n')


    # test data
    X, y = process_data(test_loader, flag=False)
    CM, f1 = evaluate(device, X, y, trained_model, batch_size=BATCH_SIZE)
    write_log('results.txt', f'Confusion Matrix:\n{CM}\nF1 Score: {f1}\n')



elif model_type == 'MLNN':
    model = DNN.DNN(n_inputs, [size_hidden_layer] * n_layers, n_classes)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # train loader is an array of tuples (X, y) where X is a batch of images and y are the corresponding labels
    # extract X and y
    # train data    
    X, y = process_data(train_loader)
    print(X.shape, y.shape) 
    write_log("main_log.txt", f"Processed training data shapes: X: {X.shape}, y: {y.shape}\n")
    loss_values_training, trained_model = fit(device, X, y, model, loss_function, optimizer, n_epochs, batch_size=BATCH_SIZE)
    write_log('loss_values_training.txt', str(loss_values_training))
    
    # evaluate data
    X, y = process_data(validation_loader)
    CM_val, f1_val = evaluate(device, X, y, trained_model, batch_size=BATCH_SIZE)
    write_log('validation_results.txt', f'Confusion Matrix:\n{CM_val}\nF1 Score: {f1_val}\n')



    # test data
    X, y = process_data(test_loader) 
    CM, f1 = evaluate(device, X, y, trained_model, batch_size=BATCH_SIZE)
    write_log('results.txt', f'Confusion Matrix:\n{CM}\nF1 Score: {f1}\n')


else:
    # model = MLP.MLP()
    pass





Epoch [1/20], Loss: 139.2199
Epoch [2/20], Loss: 81.1620
Epoch [3/20], Loss: 71.6596
Epoch [4/20], Loss: 62.1725
Epoch [5/20], Loss: 56.1907
Epoch [6/20], Loss: 50.6283
Epoch [7/20], Loss: 47.3393
Epoch [8/20], Loss: 43.9701
Epoch [9/20], Loss: 41.8174
Epoch [10/20], Loss: 39.8113
Epoch [11/20], Loss: 36.4410
Epoch [12/20], Loss: 33.6434
Epoch [13/20], Loss: 31.5893
Epoch [14/20], Loss: 29.4098
Epoch [15/20], Loss: 28.2920
Epoch [16/20], Loss: 26.1648
Epoch [17/20], Loss: 24.4867
Epoch [18/20], Loss: 23.4108
Epoch [19/20], Loss: 21.9860
Epoch [20/20], Loss: 20.8307
